# Problem Set 1 - Industrial Organization 2

#### Chase Abram and Josh Higbee

#### Set up environment (load packages, etc.).

In [1]:
# Set working directory
dir = "/Users/JoshuaHigbee/Box/2. Second Year/2. Winter Quarter - 2021/" *
      "Industrial Organization II - Hortacsu/Problem Sets/Problem Set 1/";
cd(dir);

# Load packages
using CSV, DataFrames, Random, Distributions, LinearAlgebra,
      LatexPrint, StatsBase, Plots, SpecialFunctions
using Optim, ForwardDiff, PyCall

# Set seed
Random.seed!(12345);

# Read in data
data = CSV.read("psetOne.csv", DataFrame);
println(names(data))

["Market", "Constant", "Price", "EngineSize", "SportsBike", "Brand2", "Brand3", "z1", "z2", "z3", "z4", "shares"]


<br/><br/><br/><br/><br/><br/>

## Question 8

#### Load data for market $t=17$, and set up parameter values (coefficient order is Price, Constant, Engine CC, BikeType, Brand2, Brand3) and $\xi$ values.

In [2]:
# Data 
d17 = data[data.Market .== 17, :];

# Parameters
θ_given =  [-3.0 1.0 1.0 2.0 -1.0 1.0]'
α = θ_given[1]

# Ownership matrix
d17.Brand1 = Ref(1.0) .- max.(d17.Brand2, d17.Brand3);
Δ = (d17.Brand1 .* d17.Brand1') .+ (d17.Brand2 .* d17.Brand2') .+ (d17.Brand3 .* d17.Brand3')

# ξ values
ξ_rand = rand(Normal(0,1),7);
ξ_newton = ξ_rand;

#### Define useful functions for solving fixed point pricing equation.

In [3]:
# Shares function
function s_newton(price)
    X = hcat(price, d17.Constant, d17.EngineSize, d17.SportsBike, d17.Brand2, d17.Brand3)
    exp_term = X * θ_given .+ ξ_newton
    max_exp = maximum(exp_term)
    num = exp.(exp_term .- max_exp)
    return num / (exp(-max_exp) + sum(num))
end


# Jacobian of shares function
function s_Jac_newton(price)
    s = s_newton(price)
    return - α * ((s * s') .- diagm(vec(s))) # α is negative in this setting
end


# Hessian of shares function
function s_Hess_newton(price)
    s = s_newton(price)
    Hess = Array{Float64}(undef, 7, 7, 7)
    for j in 1:7
        for k in 1:7
            for l in 1:7
                Hess[j,k,l] = α^2 *
                      ((2*s[j]*s[k]*s[l] - 3*s[j]*s[k] + s[j])*(j==k && j==l) +
                      (2*s[j]*s[k]*s[l] - s[j]*s[l])*(j==k && j!=l) +
                      (2*s[j]*s[k]*s[l] - s[k]*s[l])*(j!=k && j==l) +
                      (2*s[j]*s[k]*s[l] - s[j]*s[k])*(j!=k && k==l) +
                      (2*s[j]*s[k]*s[l])*(l!=k && k!=j && j!=l))
            end
        end
    end
    return Hess
end


# Function for Newton's method (using p + Ω^{-1} * s = 0)
function f_newton(price)
    s = s_newton(price)
    s_J = s_Jac_newton(price)
    Ω = Δ .* s_J
    f = price + inv(Ω) * s
    return f
end


# Jacobian of Ω^{-1} (for use in computing Jacobian of function for fxp est)
function Ω_inv_Jac_newton(price)
    s = s_newton(price)
    s_J = s_Jac_newton(price)
    s_H = s_Hess_newton(price)
    Ω = Δ .* s_J
    Ω_J = Δ .* s_H

    Ω_J_inv = Array{Float64}(undef, length(s), length(s), length(s))
    for j in 1:length(s)
        Ω_J_inv[:,:,j] = - inv(Ω) * Ω_J[:,:,j] * inv(Ω)
    end
    return Ω_J_inv
end


# Jacobian of function for Newton's method
function f_newton_J(price)
    s = s_newton(price)
    s_J = s_Jac_newton(price)
    Ω = Δ .* s_J
    Ω_inv_J = Ω_inv_Jac_newton(price)
    Ω_inv_J_times_s = Matrix(0.0 * I, length(s), length(s))
    for j in 1:length(s)
        Ω_inv_J_times_s[:,j] = Ω_inv_J[:,:,j] * s
    end
    ∇f_newton = I + Ω_inv_J_times_s + inv(Ω) * s_J
    return ∇f_newton
end


# Actually estimate the fixed point
function newton_fxp(price, ϵ)
    diff = 1
    p_0 = price
    iter=1
    while diff > ϵ
        # Solve equation
        fxp = f_newton(p_0)
        ∇fxp = f_newton_J(p_0)
        # ∇fxp = ForwardDiff.jacobian(f_newton, p_0)
        p_1 = p_0 - inv(∇fxp) * fxp

        # Update values and loop
        diff = maximum(abs.(p_1 .- p_0))
        println("Iteration " * string(iter) * "     Difference " * string(diff))
        p_0 = p_1
        iter = iter + 1
    end
    return p_0
end

newton_fxp (generic function with 1 method)

#### Solve fixed point equation.

In [4]:
p_init = ones(7);
p_init = d17.Price;
p_optimal = newton_fxp(p_init, 1.0e-10)

println(" ")
p_optimal

Iteration 1     Difference 3.148512235621258
Iteration 2     Difference 0.7239724490699186
Iteration 3     Difference 0.12762483506041966
Iteration 4     Difference 0.021807386009408125
Iteration 5     Difference 0.0004587246874596662
Iteration 6     Difference 1.8306169358162094e-7
Iteration 7     Difference 2.886579864025407e-14
 


7×1 Array{Float64,2}:
 0.39659106475330075
 0.3965910647533008
 0.34929855483596384
 0.3492985548359639
 1.623850783822934
 1.6238507838229337
 1.6238507838229337

In [5]:
tabular(round.(p_optimal, digits=5))

\begin{tabular}{c}
$0.39659$\\
$0.39659$\\
$0.3493$\\
$0.3493$\\
$1.62385$\\
$1.62385$\\
$1.62385$
\end{tabular}


#### Check to ensure it is a fixed point.

In [6]:
f_newton(p_optimal)
show(stdout, "text/plain", f_newton(p_optimal))

7×1 Array{Float64,2}:
 -5.551115123125783e-17
  5.551115123125783e-17
  5.551115123125783e-17
  1.1102230246251565e-16
 -1.9984014443252818e-15
 -2.220446049250313e-15
 -1.9984014443252818e-15

<br/><br/><br/><br/><br/><br/>

## Question 9

#### Write share prediction functions.

In [7]:
# Individual share predictor function (given mean utilities)
function sHat_ind(;δ, X=0.0, σ, ζ=0.0, I_n=1)
    
    # Fill values if unspecified (default)
    if X == 0.0
        X = repeat(fill(0.0,length(σ))', length(δ))
    end
    if ζ == 0.0
        ζ = reshape(fill(0.0,length(σ)*I_n), I_n, length(σ))
    end

    # Construct numerators
    const_term = repeat(δ', outer=I_n)
    rc_term = ζ * (X .* repeat(σ, outer=length(δ)))'
    exp_term = const_term .+ rc_term
    num = exp.(exp_term)

    # Construct denominators and shares
    denom = repeat(Ref(1.0) .+ sum(num, dims=2), inner=(1,length(δ)))
    ŝ_i = num ./ denom
    return ŝ_i
end

# Share predictor function (for heterogeneity)
function sHat(;δ, X=0.0, σ, ζ=0.0, I_n=1)
    ŝ_i = sHat_ind(δ=δ, X=X, σ=σ, ζ=ζ, I_n=I_n)
    ŝ = 1/(I_n) * sum(ŝ_i, dims=1)
    return vec(ŝ)
end

# Share predictor function for taking ŝ after it's already computed
function sHat_from_ŝ(; ŝ_i, I_n=1)
    ŝ = 1/(I_n) * sum(ŝ_i, dims=1)
    return vec(ŝ)
end

sHat_from_ŝ (generic function with 1 method)

#### Set up parameters and data for test cases.

In [8]:
J_q9 = [3, 3, 3];
δ_q9 = [zeros(J_q9[1]), [40, 20, 20], zeros(J_q9[3])];
σ_q9 = [[0.0], [0.0], [0.1]];
X_q9 = [[], [], [repeat([1.0], 3), [1.0; 3.0; -1.0], [10.0; 10.0; -3.0]]];
I_q9 = [[], [], 50];
ζ_q9 = [[], [], reshape(rand(Normal(0,1),length(σ_q9[3])*I_q9[3]), I_q9[3], length(σ_q9[3]))];
shares_q9 = [];

#### Test all cases.

In [9]:
# First case
s_test = sHat(δ=δ_q9[1], σ=σ_q9[1]);
println(s_test);
sum(s_test, dims=2)[1]
push!(shares_q9, s_test);

[0.25, 0.25, 0.25]


In [10]:
# Second case
s_test = sHat(δ=δ_q9[2], σ=σ_q9[2]);
println(s_test);
sum(s_test, dims=2)[1]
push!(shares_q9, s_test);

[0.9999999958776928, 2.0611536139418496e-9, 2.0611536139418496e-9]


In [11]:
# Third case
s_test = sHat(δ=δ_q9[3], X=X_q9[3][1], σ=σ_q9[3], ζ=ζ_q9[3], I_n=I_q9[3])
println(convert.(Float64,round.(s_test, digits=7)))
push!(shares_q9, s_test);

s_test = sHat(δ=δ_q9[3], X=X_q9[3][2], σ=σ_q9[3], ζ=ζ_q9[3], I_n=I_q9[3])
println(convert.(Float64,round.(s_test, digits=7)))
push!(shares_q9, s_test);

s_test = sHat(δ=δ_q9[3], X=X_q9[3][3], σ=σ_q9[3], ζ=ζ_q9[3], I_n=I_q9[3])
println(convert.(Float64,round.(s_test, digits=7)))
push!(shares_q9, s_test);

[0.2499492, 0.2499492, 0.2499492]
[0.247299, 0.2547225, 0.2504074]
[0.2492555, 0.2492555, 0.2680796]


The shares are what we would expect them to be - notably, in the second case, the specification of the logit error means that even when there are extremely large mean utilities, there will still be some nonzero market share for the less popular goods (even the outside good, with mean utility normalized to 0).

<br/><br/><br/><br/><br/><br/>

## Question 10 

#### Create functions for share inversion.

After attempting the Lipschitz constant trick, we found that convergence was faster and more dependable with a switching rule.

In [12]:
# Contraction mapping function
function s_contraction_map(;s, X, I_n, σ, ζ, J, ϵ_contract = 1.0e-5, output=false)
      δ_0 = convert.(Real, zeros(length(s)))
      iter = 0 # Initialize iteration count
      δ_diff = 0.5 # Initialize difference in δ
      δ_diff_h = Array{Real}(undef, 1, 3)
      while (δ_diff > ϵ_contract) 
            # Numerical inversion
            δ_1 = δ_0 .+ log.(s) .- log.(sHat(δ=δ_0, X=X, σ=σ, ζ=ζ, I_n=I_n))

            # Max distance in δ
            δ_diff = maximum(abs.(δ_1 - δ_0))

            if (iter % 50) == 0 && output==true
                println("Iteration  " * string(iter) * "    Difference is " * string(δ_diff))
            end
            δ_0 = δ_1
      end

      return δ_0
end


# Log shares Jacobian function - vectorized
function log_s_Jac(; ŝ_i, I_n=1, J)
      ∫ŝ_j_mat = repeat(1/(I_n) .* sum(ŝ_i, dims=1), outer=J)'
      own_term = 1.0/(I_n) * ŝ_i' * (Ref(1.0) .- ŝ_i) .* Matrix(I, J, J)
      cross_term = -1.0/(I_n) * (ŝ_i' * ŝ_i) .* (Ref(1.0) .- Matrix(I, J, J))
      return (own_term .+ cross_term) ./ ∫ŝ_j_mat
end


# Newton's method function
function s_newton_fxp(; s, δ_0, X, I_n, σ, ζ, J, ϵ_conv=1.0e-14, output=false, maxiter = 1e5)
    diff = 0.5
    iter = 0 # Initialize iteration count
    while diff > ϵ_conv && iter < maxiter
        # Compute
        ŝ_i = sHat_ind(δ=δ_0, X=X, σ=σ, ζ=ζ, I_n=I_n)
        ŝ = sHat_from_ŝ(ŝ_i=ŝ_i, I_n=I_n)
        log_sJ = log_s_Jac(ŝ_i=ŝ_i, I_n=I_n, J=J)
        log_s = log.(ŝ)
        δ_1 = δ_0 .- inv(log_sJ) * (log_s - log.(s))

        # Check difference and iterate
        diff = maximum(abs.(δ_1 .- δ_0))
        iter = iter + 1
        if output==true
              println("Iteration  " * string(iter) *
                    "    Difference is " * string(diff))
        end
        δ_0 = δ_1
    end

    return δ_0
end


# Combine two methods function
function invert_s(; s, X, I_n, σ, ζ, J, ϵ_contract=1.0e-5, ϵ_conv=1.0e-14, output=false, maxiter_newton = 1e5)
    δ_contraction = s_contraction_map(s=s, X=X, I_n=I_n, σ=σ, ζ=ζ, J=J, ϵ_contract=ϵ_contract, output=output)
    δ = s_newton_fxp(s=s, δ_0=δ_contraction, X=X, I_n=I_n, σ=σ, ζ=ζ, J=J, ϵ_conv=ϵ_conv, output=output, 
            maxiter = maxiter_newton)
    return δ
end

invert_s (generic function with 1 method)

#### Test functions with data from market 17.

In [13]:
I_17 = 25;
X_17 = convert(Matrix, d17[:, [:Price, :Constant, :EngineSize, :SportsBike, :Brand2, :Brand3]]);
Z_17 = convert(Matrix, d17[:, [:z1, :z2, :z3, :z4]]);
σ_17 = zeros(size(X_17)[2])';
ζ_17 = reshape(rand(Normal(0,1),length(σ_17)*I_17), I_17, length(σ_17));

In [14]:
# Test contraction
δ_contract = s_contraction_map(s=d17.shares, X=X_17, I_n=I_17, σ=σ_17, ζ=ζ_17, J=size(X_17)[1])

7-element Array{Float64,1}:
  2.0697845385676574
 -0.3439439517828262
 -2.9273567831528453
  0.6068451864947533
  3.141160653013621
  1.3883038447921399
  2.3472954727255972

In [15]:
# Test Newton's method
s_newton_fxp(s=d17.shares, δ_0 = δ_contract, X=X_17, I_n=I_17, σ=σ_17, ζ=ζ_17, J=size(X_17)[1], ϵ_conv=1.0e-14)

7-element Array{Float64,1}:
  2.070264979033944
 -0.34346351131653924
 -2.9268763426865583
  0.6073256269610395
  3.1416410934799073
  1.3887842852584265
  2.3477759131918825

In [16]:
# Test both
invert_s(s=d17.shares, X=X_17, I_n=I_17, σ=σ_17, ζ=ζ_17, J=size(X_17)[1], ϵ_contract=1.0e-4, ϵ_conv=1.0e-14)

7-element Array{Float64,1}:
  2.070264979033939
 -0.34346351131654446
 -2.926876342686563
  0.6073256269610349
  3.1416410934799024
  1.3887842852584218
  2.347775913191879

#### Test with data from the previous question.

In [17]:
println(invert_s(s=shares_q9[1], σ=σ_q9[2], X=0.0, I_n=1, ζ=0.0, J=J_q9[1]))
println(invert_s(s=shares_q9[2], σ=σ_q9[2], X=0.0, I_n=1, ζ=0.0, J=J_q9[2], ϵ_contract=1.0e-4))
println(invert_s(s=shares_q9[3], σ=σ_q9[3], X=X_q9[3][1], I_n=I_q9[3], ζ=ζ_q9[3], J=J_q9[3]))
println(invert_s(s=shares_q9[3], σ=σ_q9[3], X=X_q9[3][2], I_n=I_q9[3], ζ=ζ_q9[3], J=J_q9[3]))
println(invert_s(s=shares_q9[3], σ=σ_q9[3], X=X_q9[3][3], I_n=I_q9[3], ζ=ζ_q9[3], J=J_q9[3]))

[0.0, 0.0, 0.0]
[38.46485052094403, 18.464850520944033, 18.464850520944033]
[0.0, 0.0, 0.0]
[0.00016785577094455723, -0.029659081493348612, -0.012089398619323165]
[-0.0611761787212328, -0.0611761787212328, -0.14113460280480175]


The inversion performs well, though not perfectly.  The second case (where mean utilities were 40 for the first good and 20 for the others) shows the biggest discrepancy, but even there, the differences are relatively small.

<br/><br/><br/><br/><br/><br/>

## Question 11

#### Write function for computing $\xi$.

In [18]:
# Implicit function for ξ - all markets (X_all is vector of matrices)
function ξ_rclogit(; X_all, θ, s_all, ζ=0.0, I_num=1, maxiter_newton=1e5, rclogit_output=false)
    # θ split between linear terms (1) and nonlinear terms (2)
    αβ = θ[1] # Put price first for consistency
    σ = θ[2]
    
    # Loop through markets and invert shares
    δhat_all = []
    for mkt in 1:length(X_all)
        if rclogit_output==true
            println("Market number is " * string(mkt) * " of " * string(length(X_all)))
        end
        δhat = invert_s(s=s_all[mkt], X=X_all[mkt], I_n=I_num, σ=σ, ζ=ζ, J=length(s_all[mkt]), 
                    ϵ_contract=1.0e-3, ϵ_conv=1.0e-14, maxiter_newton = maxiter_newton)
        push!(δhat_all, δhat)
    end
    
    # Stack data and compute δ_mean
    δhat = reduce(vcat, δhat_all)
    X_stack = reduce(vcat, X_all)
    δ_mean = X_stack * αβ

    # Subtract off mean utility to get ξ̂
    return δhat .- δ_mean
end

ξ_rclogit (generic function with 1 method)

#### Write function for splicing data into markets (array of matrices and vectors, for shares and variables).

In [19]:
function mkt_X_s_break(; exclude_vec)
    X_full_mkt = []
    s_full_mkt = []
    for mkt in unique(data[[x ∉ exclude_vec for x in data.Market], :Market])
        X_mkt = convert(Matrix, data[data.Market .== mkt,
                    [:Price, :Constant, :EngineSize, :SportsBike, :Brand2, :Brand3]])
        push!(X_full_mkt, X_mkt)
        
        s_mkt = data[data.Market .== mkt, :shares]
        push!(s_full_mkt, s_mkt)
    end
    return [X_full_mkt, s_full_mkt]
end

mkt_X_s_break (generic function with 1 method)

#### Test functions with data from market.

For testing purposes, we used Z_test and W_test for using only instruments z1-z4, while Z_full and W_full included the non-price covariates (for identification purposes, these are eventually preferred in the estimation process, but having both allowed for more test cases to check the functions).

In [20]:
# Choose data
test_data = mkt_X_s_break(exclude_vec = []);

# Create test parameters and select data
I_test = 25
σ_test = zeros(6)
ζ_test = reshape(rand(Normal(0,1),length(σ_test)*I_test), I_test, length(σ_test))
Z_test = convert(Matrix, data[:, [:z1, :z2, :z3, :z4]])
W_test = inv(Z_test' * Z_test);


# Test with expanded instrument set
X_no_p = reduce(vcat, test_data[1])[:,2:6]
Z_full = hcat(X_no_p, Z_test)
W_full = inv(Z_full' * Z_full)

9×9 Array{Float64,2}:
  0.0284132    -0.00237115   -0.0126792    …   0.00152733   -0.000417313
 -0.00237115    0.000479658   0.00100226      -0.00109375   -0.00100517
 -0.0126792     0.00100226    0.00809334      -0.000931723   0.000182673
 -0.00771592    0.00014338    0.00276867      -0.000362551   0.000154876
 -0.00332249   -0.000369468   0.000845524     -0.000244754   0.00055841
  0.00149303   -0.0011736    -0.000803375  …   0.000968919   0.000398021
  0.000499247  -0.000975531  -0.000572778     -0.00025748   -0.00036107
  0.00152733   -0.00109375   -0.000931723      0.0203565    -0.000652484
 -0.000417313  -0.00100517    0.000182673     -0.000652484   0.0203369

In [21]:
# Test functions at θ = 0
θ_zeros = [zeros(6), zeros(6)']
@time ξ_zeros = ξ_rclogit(X_all=test_data[1], θ=θ_zeros, s_all=test_data[2], ζ=ζ_test, I_num=I_test);

 33.064563 seconds (138.92 M allocations: 21.013 GiB, 18.63% gc time)


In [22]:
ξ_zeros[1:8]

8-element Array{Float64,1}:
  3.4175577069186134
  2.219989832335631
 -2.834820623687745
  4.317272874619497
  1.4596209841341568
  1.0254328494996454
  1.5368650848736907
 -2.7318744786927485

#### Write GMM objective function using implicit function $\xi$.

In [23]:
function f_gmm_rclogit_implθ(; W=I, Z, X_all, θ, s_all, ζ, I_num)
    ξ = ξ_rclogit(X_all=X_all, θ=θ, s_all=s_all, ζ=ζ, I_num=I_num)
    return (Z' * ξ)' * W * (Z' * ξ)
end

# Test objective function - computing ξ within (only z1-z4)
f_gmm_rclogit_implθ(W=W_test, Z=Z_test, X_all=test_data[1], θ=[zeros(6), zeros(6)'], s_all=test_data[2], 
    ζ=ζ_test, I_num=I_test)

252.5932803130469

In [24]:
# Test objective function - computing ξ within (using full instruments, including characteristics)
f_gmm_rclogit_implθ(W=W_full, Z=Z_full, X_all=test_data[1], θ=[zeros(6), zeros(6)'], s_all=test_data[2], 
    ζ=ζ_test, I_num=I_test)

2947.137997931582

#### Rewrite objective function as a function of only $\sigma$ (no linear terms).

This still involves writing the GMM objective as a function of $\xi$, but it limits the parameters used by $\xi$ to reduce the dimensionality of the problem and avoid searching over linear terms that can be computed directly.

In [25]:
# Function for computing linear parameters
function θ_tsls(; X, Z, W, δ)
    B = X' * Z * W * Z'
    return inv(B * X) * (B * δ)
end


# Function returning ξ(σ), δ(σ), and θ̄(σ)
function ξδθ_σ_rclogit(; X_all, σ, s_all, ζ=0.0, I_num=1, Z, W)
    δ = []
    for mkt in 1:length(X_all)
        δ_mkt = invert_s(s=s_all[mkt], X=X_all[mkt], I_n=I_num, σ=σ, ζ=ζ, J=length(s_all[mkt]))
        push!(δ, δ_mkt)
    end
    δ_σ_stack = reduce(vcat, δ)
    X_stack = reduce(vcat, X_all)

    αβ = θ_tsls(X=X_stack, Z=Z, W=W, δ=δ_σ_stack)
    
    ξ_all = []
    for mkt in 1:length(X_all)
        ξ_mkt = δ[mkt] .- X_all[mkt] * αβ
        push!(ξ_all, ξ_mkt)
    end
    ξ = reduce(vcat, ξ_all)
    
    return (ξ = ξ, δ_all = δ, αβ = αβ, ξ_all = ξ_all)
end


# Rewrite objective function as only a function of σ
function f_gmm_rclogit_σ(;W, Z, X_all, σ, s_all, ζ, I_num)
    output = ξδθ_σ_rclogit(X_all=X_all, σ=σ, s_all=s_all, ζ=ζ, I_num=I_num, Z=Z, W=W)
    ξ = output.ξ
    # println(output.αβ)
    return (Z' * ξ)' * W * (Z' * ξ)
end

f_gmm_rclogit_σ (generic function with 1 method)


#### Now, we check the value of the objective function when $\sigma$ is constrained to be 0 (the value is lower since the linear parameters are unconstrained and thus optimally calculated).

In [26]:
@time f_gmm_rclogit_σ(W=W_full, Z=Z_full, X_all=test_data[1], σ=zeros(6)', s_all=test_data[2], 
    ζ=ζ_test, I_num=I_test)

 14.470049 seconds (81.91 M allocations: 12.894 GiB, 20.01% gc time)


0.7241370155056207

<br/><br/><br/><br/><br/><br/>

## Question 12

#### Write gradient function (as a function of $\sigma$ only).

In [27]:
# Individual share predictor function (given X and αβ)
function sHat_ind_αβ(; αβ, X, ξ, σ, ζ=0.0, I_n=1)
    # Construct numerators
    δ = X * αβ .+ ξ
    const_term = repeat(δ', outer=I_n)
    rc_term = ζ * (X .* repeat(σ, outer=length(δ)))'
    exp_term = const_term .+ rc_term
    num = exp.(exp_term)

    # Construct denominators and shares
    denom = repeat(Ref(1.0) .+ sum(num, dims=2), inner=(1,length(δ)))
    ŝ_i = num ./ denom
    return ŝ_i
end


# Function to compute Jacobian of ξ(θ) for each market
function ξ_Jac_θ_mkt(; X, I_n, ζ, ŝ_i)
    ξ_J = zeros(size(X)[1], size(X)[1])
    J_σ = zeros(size(X)[1], size(X)[2])
    for i in 1:I_n
        s = ŝ_i[i,:]
        ξ_Ji = -1.0 * ((s * s') .- diagm(vec(s)))
        ξ_J = ξ_J .+ (1/I_n) .* ξ_Ji
        J_σ = J_σ .+ (ξ_Ji * X) * diagm(ζ[i,:])
    end
    return -1.0 * inv(ξ_J) * J_σ./I_n
end


# Function for computing Jacobian of ξ(θ) for all markets
function ξ_Jac(; X_all, αβ, I_n, ζ, σ, ξ_all)
      ξ_J_θ = []
      for mkt in 1:length(X_all)
            X_mkt = X_all[mkt];
            ŝ_i = sHat_ind_αβ(αβ=αβ, X=X_mkt, ξ=ξ_all[mkt], σ=σ, ζ=ζ, I_n=I_n)
            ξ_J_θ_mkt = ξ_Jac_θ_mkt(X=X_mkt, I_n=I_n, ζ=ζ, ŝ_i=ŝ_i)
            push!(ξ_J_θ, ξ_J_θ_mkt)
      end
      ξ_J = reduce(vcat, ξ_J_θ)
      return ξ_J
end

ξ_Jac (generic function with 1 method)

#### Write functions that compute analytic gradient (and objective function as a function of $\sigma$ only, for comparison with ForwardDiff).

In [28]:
# Function that computes gradient given σ and only σ
function f_gmm_rclogit_σ_g(; X_all, s_all, Z, W, σ, I_n, ζ)
    ξδθ_output =  ξδθ_σ_rclogit(X_all=X_all, σ=σ, s_all=s_all, ζ=ζ, I_num=I_n, Z=Z, W=W);
    ξ_J = ξ_Jac(X_all=X_all, αβ=ξδθ_output.αβ, I_n=I_n, ζ=ζ, σ=σ, ξ_all=ξδθ_output.ξ_all)
    df_θ = 2 * (Z' * ξ_J)' * W * (Z' * ξδθ_output.ξ)
    return df_θ
end

# Function that computes objective function for given σ, using full data
function gmm_σ_test_full(σ_init) 
    func = f_gmm_rclogit_σ(W=W_full, Z=Z_full, X_all=test_data[1], σ=σ_init,
                        s_all=test_data[2], ζ=ζ_test, I_num=I_test)
    return func
end

# Function that computes objective function for given σ, using "test" data
function gmm_σ_test(σ_init) 
    func = f_gmm_rclogit_σ(W=W_test, Z=Z_test, X_all=test_data[1], σ=σ_init,
                        s_all=test_data[2], ζ=ζ_test, I_num=I_test)
    return func
end

gmm_σ_test (generic function with 1 method)

#### Compare auto-differentiation with analytic gradients, using the full set of instruments.

##### At 0:

In [29]:
# Analytic
f_gmm_rclogit_σ_g(X_all=test_data[1], s_all=test_data[2], Z=Z_full, W=W_full, 
    σ=zeros(6)', I_n=I_test, ζ=ζ_test)'

1×6 Adjoint{Float64,Array{Float64,1}}:
 3.68239e-12  4.60724e-12  9.63633e-12  …  -5.34452e-13  -3.61953e-12

In [30]:
# Auto-diff
ForwardDiff.gradient(gmm_σ_test_full, zeros(6)')

1×6 Adjoint{Float64,Array{Float64,1}}:
 1.69648e-15  4.31271e-15  -1.20981e-14  …  1.11566e-15  -1.12301e-15

##### At 0.1:

In [31]:
# Analytic
f_gmm_rclogit_σ_g(X_all=test_data[1], s_all=test_data[2], Z=Z_full, W=W_full, 
    σ=0.1*ones(6)', I_n=I_test, ζ=ζ_test)'

1×6 Adjoint{Float64,Array{Float64,1}}:
 0.128023  0.00753761  0.988349  0.0386226  0.136681  0.147409

In [32]:
# Auto-diff
ForwardDiff.gradient(gmm_σ_test_full, 0.1*ones(6)')

1×6 Adjoint{Float64,Array{Float64,1}}:
 0.128023  0.00753761  0.988349  0.0386226  0.136681  0.147409

While the values do not match at zero, this may be more a result of them being very small and there being differences in computation - note that even at $\sigma = 0.1$, the analytic and auto-differentiated gradients match each other well.

<br/><br/><br/><br/><br/><br/>

## Question 13

#### Write GMM function that only takes in the desired values of $\sigma$ (for random coefficients on price and engine size).

##### GMM function using full sets of instruments.

In [33]:
function gmm_σ_full(σ_init) 
    σ_mod = [σ_init[1], 0, σ_init[2], 0, 0, 0]' # Only price and engine size have RCs
    func = f_gmm_rclogit_σ(W=W_full, Z=Z_full, X_all=test_data[1], σ=σ_mod,
                        s_all=test_data[2], ζ=ζ_test, I_num=I_test)
    return func
end

gmm_σ_full (generic function with 1 method)

In [34]:
# Test GMM function
gmm_σ_full(0.1*ones(2)')

0.7637524786013782

In [35]:
function gmm_σ_full_g(σ_init) 
    σ_mod = [σ_init[1], 0, σ_init[2], 0, 0, 0]' # Only price and engine size have RCs
    grad = f_gmm_rclogit_σ_g(X_all=test_data[1], s_all=test_data[2], Z=Z_full, W=W_full, 
                σ=σ_mod, I_n=I_test, ζ=ζ_test)'
    return grad[[1,3]]
end

gmm_σ_full_g (generic function with 1 method)

In [36]:
gmm_σ_full_g(0.1*ones(2)')

2-element Array{Float64,1}:
 0.14836765194144996
 0.7502971061067116

##### One more sanity check to make sure the gradients match.

In [37]:
# Construct auto-diff function as placeholder
g_gmm_σ_auto_full = x -> ForwardDiff.gradient(gmm_σ_full, x)
g_gmm_σ_auto_full(0.1 * ones(2)')

1×2 Adjoint{Float64,Array{Float64,1}}:
 0.148368  0.750297

#### Write function for calling one stage of GMM for a given weighting matrix.

In [38]:
function gmm_stage(; W, Z, X_all, s_all, ζ, I_num, σ_init)
    # Objective function
    function f_obj(σ_init) 
        σ_mod = [σ_init[1], 0, σ_init[2], 0, 0, 0]' # Only price and engine size have RCs
        func = f_gmm_rclogit_σ(W=W, Z=Z, X_all=X_all, σ=σ_mod, s_all=s_all, ζ=ζ, I_num=I_num)
        return func
    end
    
    # Gradient function
    function g_obj(G, σ_init)
        grad = gmm_σ_full_g(σ_init)
        G[1:2] = grad
    end
    
#     # AD gradient function - for testing Optim and comparison with coded gradient function
#     g_gmm_σ_auto_test = x -> ForwardDiff.gradient(f_obj, x)
#     function g_obj(G, σ_init)
#         grad = g_gmm_σ_auto_test(σ_init)
#         G[1:2] = grad[1:2]
#     end
    
    # Return result
    σ_result = Optim.minimizer(optimize(f_obj, g_obj, σ_init, BFGS(), Optim.Options(show_trace=true,
                x_tol=1e-6, f_tol=1e-6, g_tol=1e-4)))
    return σ_result
end  

gmm_stage (generic function with 1 method)

#### Write functions for implementing two-stage GMM.

In [39]:
# Function for optimal weighting matrix
function gmm_opt_weight(; Z, ξ)
    n = size(Z)[1]
    Zξ = Z' * ξ
    return n * inv(Z' * ξ * ξ' * Z)
end


# Function for gradient of ξ with respect to all parameters
function ZJ_θfull_J(; Z, ξ, X_all, αβ, I_n, ζ, σ)
    n = size(Z)[1]
    Jac_αβ = -1 * reduce(vcat, X_all)
    Jac_σ = ξ_Jac(X_all=X_all, αβ=αβ, I_n=I_n, ζ=ζ, σ=σ, ξ_all=ξ)
    Jac_θ = hcat(Jac_αβ, Jac_σ)
    return (1/n) * Z' * Jac_θ
end


# Function for two-stage GMM
function two_stage_gmm(; W_stage1, Z, X_all, s_all, ζ, I_num, σ_init)
    
    # Print feedback
    println("Beginning Stage 1 of GMM - - - - - - - - - - - - - - - - - - - - - - - -")
    
    # First stage estimation with given starting weighting matrix
    σ_stage1 = gmm_stage(W=W_stage1, Z=Z, X_all=X_all, s_all=s_all, ζ=ζ, I_num=I_num, σ_init=σ_init)
    
    # Compute optimal weighting matrix for second stage
    σ_mod = [σ_stage1[1], 0, σ_stage1[2], 0, 0, 0]'
    ξδθ_output =  ξδθ_σ_rclogit(X_all=X_all, σ=σ_mod, s_all=s_all, ζ=ζ, I_num=I_num, Z=Z, W=W_stage1)
    ξ_stage1 = ξδθ_output.ξ
    W_stage2 = gmm_opt_weight(Z=Z, ξ=ξ_stage1)
    
    # Print feedback
    println("Beginning Stage 2 of GMM - - - - - - - - - - - - - - - - - - - - - - - -")
    
    # Second stage estimation with given starting weighting matrix
    σ_stage2 = gmm_stage(W=W_stage2, Z=Z, X_all=X_all, s_all=s_all, ζ=ζ, I_num=I_num, σ_init=σ_init)
    
    # Compute covariance matrix for point estimates
    σ_mod = [σ_stage2[1], 0, σ_stage2[2], 0, 0, 0]'
    ξδθ_output =  ξδθ_σ_rclogit(X_all=X_all, σ=σ_mod, s_all=s_all, ζ=ζ, I_num=I_num, Z=Z, W=W_stage2)
    G = ZJ_θfull_J(Z=Z, ξ=ξδθ_output.ξ_all, X_all=X_all, αβ=ξδθ_output.αβ, I_n=I_num, ζ=ζ, σ=σ_mod)
    Σ = inv(G' * inv(W_stage2) * G)
    
    # Print feedback
    println("Converged - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -")

    # Return values
    return (σ = σ_stage2, αβ = ξδθ_output.αβ, Σ = Σ, W = W_stage2)
end

two_stage_gmm (generic function with 1 method)

#### Call GMM function to estimate objective.

In [ ]:
results = two_stage_gmm(W_stage1=W_full, Z=Z_full, X_all=test_data[1], s_all=test_data[2], 
    ζ=ζ_test, I_num=I_test, σ_init=ones(2))

Beginning Stage 1 of GMM - - - - - - - - - - - - - - - - - - - - - - - -
Iter     Function value   Gradient norm 
     0     1.712143e+01     3.882115e+01
 * time: 0.00038909912109375
     1     4.072115e+00     8.131581e+00
 * time: 68.70479321479797
     2     2.354170e+00     1.027979e+01
 * time: 113.33254599571228
     3     8.148378e-01     1.283586e+00
 * time: 154.97151017189026
     4     7.291950e-01     1.026447e-01
 * time: 204.94468021392822
     5     7.255223e-01     7.338306e-02
 * time: 266.62931513786316


In [ ]:
tabular(reshape(round.(results.σ, sigdigits=4), 2, 1))

In [ ]:
tabular(reshape(round.(results.αβ, sigdigits=4), 6,1))

In [ ]:
tabular(reshape(diag(round.(abs.(results.Σ)/size(Z_full, 1), sigdigits=4)), 12, 1))

<br/><br/><br/><br/><br/><br/><br/><br/>

## Question 14

##### Use PyBLP

In [ ]:
pyblp = pyimport("pyblp")
np = pyimport("numpy")
pd = pyimport("pandas")
os = pyimport("os")

In [ ]:
os.getcwd()

In [ ]:
product_data = pd.read_csv("psetOne.csv")

In [ ]:
product_data.rename(columns=Dict("Price"=>"prices", "Market"=>"market_ids", "z1"=>"demand_instruments0",
    "z2"=>"demand_instruments1", "z3"=>"demand_instruments2", "z4"=>"demand_instruments3"), 
    inplace="True")

In [ ]:
product_formulations = (pyblp.Formulation("prices + 1 + EngineSize + SportsBike + Brand2 + Brand3"),
   pyblp.Formulation("0 + prices + EngineSize"))

In [ ]:
mc_integration = pyblp.Integration("monte_carlo", size=25, specification_options=Dict("seed"=>12345))

In [ ]:
problem = pyblp.Problem(product_formulations, product_data, integration=mc_integration, add_exogenous="True")

In [ ]:
bfgs = pyblp.Optimization("bfgs", Dict("gtol"=>1e-4))

In [ ]:
results = problem.solve(sigma=np.identity(2), optimization=bfgs)

In [ ]:
αβ_pyblp = results.beta

In [ ]:
tabular(round.(reshape(αβ_pyblp, 6, 1), sigdigits=4))

In [ ]:
σ_pyblp = diag(results.sigma)

In [ ]:
tabular(round.(reshape(σ_pyblp, 2, 1), sigdigits=4))

<br/><br/><br/><br/><br/><br/><br/><br/>

## Question 15

#### Elasticity matrix for market $t=17$.

#### Get predicted $\xi$ from the parameters found earlier

In [ ]:
# Individual share predictor function (given X and αβ)
function sHat_ind_elast(; αβ, X, ξ, σ, ζ=0.0, I_n=1)
    # Construct numerators
    δ = X * αβ .+ ξ
    const_term = repeat(δ', outer=I_n)
    rc_term = ζ * (X .* repeat(σ', outer=length(δ)))'
    exp_term = const_term .+ rc_term
    num = exp.(exp_term)

    # Construct denominators and shares
    denom = repeat(Ref(1.0) .+ sum(num, dims=2), inner=(1,length(δ)))
    ŝ_i = num ./ denom
    return ŝ_i
end

In [ ]:
function elasticity(; αβ, X, σ, ζ, ξ, prices)
    # Get necessary numbers
    J = size(X,1)
    el = zeros(J,J)
    n = size(ζ,1)
    
    # Predict individual and aggregate shares of each good
    ŝ_i = sHat_ind_elast(αβ=αβ, X=X, ξ=ξ, σ=σ, ζ=ζ, I_n=I_test)
    s_mkt = mean(ŝ_i, dims=1)[1,:]
    
    # Vector of individual price coefficients
    α_i = Ref(αβ[1]) .+ ζ[:,1] * σ[1]
    
    # Fill in elasticity matrix
    for j in 1:J
        for k in 1:J
            if j==k
                el[j,k] = - (prices[j]/s_mkt[j]) * mean(α_i[i,1] .* ŝ_i[i,j]*(1-ŝ_i[i,j]) for i in 1:n)
            else
                el[j,k] = (prices[k]/s_mkt[j]) * mean(α_i[i,1] .* ŝ_i[i,j]*ŝ_i[i,k] for i in 1:n)
            end
        end
    end
    return el
end

In [ ]:
σ_pyblp_all = [σ_pyblp[1], 0, σ_pyblp[2], 0, 0, 0]
αβ_pyblp_all = [αβ_pyblp[2], αβ_pyblp[1], αβ_pyblp[3:6]]

output = ξδθ_σ_rclogit(X_all=test_data[1], σ=σ_pyblp_all', s_all=test_data[2], ζ=ζ_test, I_num=I_test,
    Z=Z_full, W=W_full)

ξ_17 = output.ξ_all[17];

In [ ]:
Elast = elasticity(αβ=αβ_pyblp, X=X_17, σ=σ_pyblp_all, ζ=ζ_test, ξ=ξ_17, prices=X_17[:,1])

In [ ]:
tabular(round.(Elast, digits=5))